In [1]:
import pandas as pd

In [2]:
import os
if os.environ.get("AITUEDATA"):
    print('set')
    data_directory = os.environ['AITUEDATA']
else:
    print('not set')
    #Eusebio
#     data_directory = "C:/Users/joaquin/Desktop/mem_aitue/Datos/"
    #Mini-Eusebio
#     data_directory = "C:/Users/jquin/Desktop/Memoria de Titulo/Aitue/Datos/"
    #PC-Mem
    data_directory = "C:/Users/Joaquin/Desktop/mem_aitue/Datos/"

not set


# 1.Select Data

In [3]:
cot_all = pd.read_excel(data_directory+"cotizaciones_all.xlsx")
neg_all = pd.read_excel(data_directory+"negocios_all.xlsx")

In [4]:
cot_all.columns

Index(['ID', 'Fecha Cotizacion', 'Medio', 'Tipo de Medio', 'Proyecto', 'Etapa',
       'Productos', 'Presencial', 'Remoto', 'Jefe Comercial',
       'Ejecutivo Comercial', 'Total Productos', 'Descuentos',
       'Valor Final Venta', 'RUT', 'Nombre', 'Apellido 1', 'Apellido 2',
       'Nombre Completo', 'Telefono', 'Celular', 'Direccion', 'Region',
       'Provincia', 'Comuna', 'Correo Electronico', 'Sexo', 'Estado Civil',
       'Rango Edad', 'Fecha Nacimiento', 'Tipo Cliente', 'Razon Social',
       'Giro', 'Nacionalidad', 'N° Grupo Familiar', 'Actividad', 'Cargo',
       'Situacion Laboral', 'Empleador', 'Antiguedad Laboral', 'Profesión'],
      dtype='object')

In [50]:
s_columns = ['ID','Nombre', 'Fecha Cotizacion', 'Medio', 'Proyecto', 'Etapa','Productos', 'Presencial', 'Jefe Comercial',
             'Ejecutivo Comercial',
             'Total Productos','Descuentos','Valor Final Venta', 'RUT', 'Nombre', 'Apellido 1', 'Apellido 2',
             'Nombre Completo', 'Telefono', 'Celular', 'Direccion', 'Region','Provincia', 'Comuna', 
             'Correo Electronico', 'Sexo', 'Estado Civil','Rango Edad', 'Fecha Nacimiento', 'Profesión','Actividad']

cot_mod = cot_all[s_columns]

In [51]:
# ruts de los clientes en negocio
neg_ids = neg_all['ID Cotizacion'].unique().tolist()

# cot_neg = cot_all.copy()
cot_mod['Negocio'] = cot_all['ID'].isin(neg_ids)
cot_mod[cot_mod['Negocio'] == True].shape

C:\Users\Joaquin\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


(2336, 32)

# 2.Clean Data

In [7]:
# variables a impiar y chequear

# Limpiar variable fecha de nacimiento de alguna manera
# Limpiar variable Sexo usando rutificador
# Limpar variables Direccion usando en lo posible el rutificador y otras fuentes
# Llenar mv en decuentos con 0

In [52]:
import chile_rut
import re

def val_rut(x):
    x = str(x)
    if len(x) == 0:
        return False
    x = re.sub('[|!@#$]', '', x)
#     print(x)
    
    if len(x.split('-')) >= 2:
#         print(len(x.split('-')))
        return chile_rut.validate_rut(x)
    else:
#         print(len(x.split('-')))
        x = x[:-1] + '-' + x[-1]
        return chile_rut.validate_rut(x)

def clean_rut(x):
    x = str(x)
    
    if len(x) == 0:
        return False
    x = re.sub('[|!@#$]', '', x)
    
    if len(x.split('-')) >= 2:
        return x
    else:
        x = x[:-1] + '-' + x[-1]
        return x
    
cot_mod['Valid Rut'] = [val_rut(x) for x in cot_mod['RUT'].values]
cot_mod['Clean Rut'] = [clean_rut(x) for x in cot_mod['RUT'].values]

C:\Users\Joaquin\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Joaquin\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [69]:
#Eliminar 1704 y arreglar rut max boris
cot_mod[(cot_mod['Valid Rut'] == False) & (cot_mod['Negocio'] == True)][['Nombre','RUT','Negocio','Valid Rut','Clean Rut']]

,Nombre,Nombre,RUT,Negocio,Valid Rut,Clean Rut
7390,max boris,max boris,14030274-6,True,False,14030274-6
7504,max boris,max boris,14030274-6,True,False,14030274-6
1704,Error,Error,15516786-8,True,False,15516786-8


In [61]:
# ver que ruts invalidos pueden ser rescatados o arreglados
cot_mod[(cot_mod['Valid Rut'] == False)][['RUT','Clean Rut','Valid Rut']].shape

(69, 3)

In [66]:
cot_mod[(cot_mod['RUT'] != cot_mod['Clean Rut'])][['RUT','Clean Rut','Valid Rut']].shape

(5, 3)

In [67]:
cot_mod[(cot_mod['RUT'] == cot_mod['Clean Rut'])][['RUT','Clean Rut','Valid Rut']].shape

(16587, 3)

# 3.Construct Data

## 3.1 Nuevos Atributos

### 3.1.1.Edad
Variable construida a partir de la fecha de nacimiento

In [10]:
# Construir variable 'Edad' a partir de fecha de Nacimiento
import datetime
now = datetime.datetime.now().year

fechas_nacimiento = pd.to_datetime(cot_all['Fecha Nacimiento']).apply(lambda x: x.date()).tolist()

def anio_nac(x):
    try:
        year = int(x.year)
        dif = now - year
        if dif >= 18:
            return dif
        else:
            return 0
    except (ValueError,AttributeError):
        return 0
        
    return r

edad = [anio_nac(x) for x in fechas_nacimiento]
cot_mod['edad'] = edad



C:\Users\Joaquin\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [11]:
cot_mod = cot_mod.drop(['Fecha Nacimiento'], axis=1)

### 3.1.2.Descuento
Variable que indica si el cliente recibio descuento

In [12]:
# Construir 'descuento': binario : [True, False]
recibio_descuento = cot_all['Descuentos'].tolist()

def is_desc(x):
    try:
        desc = int(x)
        if desc > 0:
            return True
        else:
            return False
    except ValueError:
        return False

descuento = [is_desc(x) for x in recibio_descuento]
cot_mod['recibio_descuento'] = descuento
# descuento

### 3.1.3.Cantidad de Productos

In [13]:
# Construir 'productos': categorico : ['depto','casa','estacionamiento','bodega','Estudio']
productos = cot_all['Productos'].tolist()
depto = []; estacionamiento=[]; bodega = []; estudio = [];nan = 0
def get_products(x,word,numeric=False):
    w=0
    try:
        p =str(x)
        arr_p = x.split(',')
        for e in arr_p:
            if word in e:
                w+=1
#                 print(word + " :",e)
                break
            elif numeric and ('B' not in e or 'Est' not in e):
                try:
                    int(e)
#                     print(word + " :",e)
                    w+=1
                except ValueError:
                    pass
        return w
    except (ValueError,AttributeError):
        try:
            int(x)
            if numeric:
                return 1
            else:
                return 0
        except ValueError:
            return 0

cot_mod['#vivienda'] = [get_products(x,'T',numeric=True) for x in productos]
cot_mod['#bodega'] = [get_products(x,'Bod') for x in productos]
cot_mod['#estacionamiento'] = [get_products(x,'Est') for x in productos]
cot_mod['#estudio'] = [get_products(x,'Estudio') for x in productos]
cot_mod['#lan'] = [get_products(x,'lan') for x in productos]

In [14]:
print(cot_mod[['Productos','#vivienda','#bodega','#estacionamiento','#estudio','#lan']].head())
print(cot_mod[['Productos','#vivienda','#bodega','#estacionamiento','#estudio','#lan']].tail())

              Productos  #vivienda  #bodega  #estacionamiento  #estudio  #lan
0         TB-104,Est-65          1        0                 1         0     0
1  TA-302,Est-41,Bod-02          1        1                 1         0     0
2         TA-303,Est-01          1        0                 1         0     0
3  TA-302,Est-41,Bod-06          1        1                 1         0     0
4         TA-604,Est-01          1        0                 1         0     0
        Productos  #vivienda  #bodega  #estacionamiento  #estudio  #lan
2912  703,Est-003          1        0                 1         0     0
2913          912          1        0                 0         0     0
2914  Est-002,703          1        0                 1         0     0
2915  912,Est-002          1        0                 1         0     0
2916          912          1        0                 0         0     0


In [15]:
cot_mod = cot_mod.drop(['Productos'], axis=1)

### 3.1.4.Latitud y Longitud

In [16]:
# Constuir 'latitud' a partir de 'Direccion'
# Constuir 'longitud' a partir de 'Direccion'

### 3.1.6.Region
Esta variable indica si el cliente facilito el correo en alguna cotizacion

In [17]:
cot_mod['Region'].unique()

array(['VIII Región del Bío-Bío', 'XIII Región Metropolitana de Santiago',
       'VII Región del Maule', 'II Región de Antofagasta',
       'IV Región de Coquimbo', 'X Región de Los Lagos',
       'XII Región de Magallanes y de La Antártica Chilena',
       'V Región de Valparaíso',
       'VI Región del Libertador General Bernardo O Higgins',
       'XIV Región de Los Ríos',
       'XI Región de Aysen del General Carlos Ibáñez del Campo',
       'I Región de Tarapacá', 'IX Región de La Araucanía',
       'III Región de Atacama', 'XV Region de Arica y Parinacota'],
      dtype=object)

In [18]:
cot_mod['region'] = cot_all['Region'].copy()
cot_mod['region'] = cot_mod['region'].replace(
    {'VIII Región del Bío-Bío':'Bio-Bio',
     'XIII Región Metropolitana de Santiago':'Metropolitana'}
)
cot_mod['region'] = cot_mod['region'].replace(
    ['II Región de Antofagasta',
       'IV Región de Coquimbo', 'VII Región del Maule',
       'X Región de Los Lagos',
       'V Región de Valparaíso',
       'VI Región del Libertador General Bernardo O Higgins',
       'XIV Región de Los Ríos',
       'XI Región de Aysen del General Carlos Ibáñez del Campo',
       'I Región de Tarapacá', 'IX Región de La Araucanía',
       'XII Región de Magallanes y de La Antártica Chilena',
       'XV Region de Arica y Parinacota', 'III Región de Atacama'],'Otro'
)

In [19]:
cot_mod['region'] = cot_mod['region'].astype('category')
cot_mod['region'].head()

0    Bio-Bio
1    Bio-Bio
2    Bio-Bio
3    Bio-Bio
4    Bio-Bio
Name: region, dtype: category
Categories (3, object): [Bio-Bio, Metropolitana, Otro]

In [20]:
cot_mod = cot_mod.drop(['Region'], axis=1)

### 3.1.7.Provincia
Esta variable indica si el cliente facilito el correo en alguna cotizacion

In [21]:
cot_all['Provincia'].unique()

array(['Concepción', 'Ñuble', 'Santiago', 'Talca', 'Biobío', 'Maipo',
       'Antofagasta', 'Limarí', 'Llanquihue', 'Magallanes', 'El Loa',
       'Isla de Pascua', 'Valparaíso', 'Cachapoal', 'Arauco',
       'Cordillera', 'Valdivia', 'Coyhaique', 'Linares', 'Osorno',
       'San Felipe de Aconcagua', 'Iquique', 'Cautín', 'Capitán Prat',
       'Aysén', 'Chacabuco', 'Copiapó', 'Melipilla', 'Malleco', 'Arica',
       'Elqui', 'Cauquenes', 'Colchagua', 'Última Esperanza', 'Curicó',
       'Chiloé', 'Choapa', 'Huasco'], dtype=object)

In [22]:
cot_mod['provincia'] = cot_all['Provincia'].copy()
cot_mod['provincia'] = cot_mod['provincia'].replace(
    {'Concepción':'Concepcion',
     'Santiago':'Santiago'}
)
cot_mod['provincia'] = cot_mod['provincia'].replace(
    ['Antofagasta', 'Limarí', 'Talca', 'Biobío', 'Ñuble',
       'Llanquihue','Maipo', 'Valparaíso', 'Cachapoal',
       'Arauco', 'Valdivia', 'Coyhaique', 'Linares', 'Osorno',
       'San Felipe de Aconcagua', 'Iquique', 'Cautín', 'Capitán Prat',
       'Aysén', 'Chacabuco', 'Melipilla', 'Magallanes', 'Malleco',
       'Arica ', 'Elqui', 'El Loa', 'Cauquenes', 'Colchagua', 'Valdivia ',
       'Copiapó', 'Última Esperanza', 'Cordillera', 'Curicó', 'Chiloé',
       'Choapa', 'Huasco','Arica','Isla de Pascua'],'Otro'
)

In [23]:
cot_mod['provincia'] = cot_mod['provincia'].astype('category')
cot_mod['provincia'].head()

0    Concepcion
1    Concepcion
2    Concepcion
3    Concepcion
4          Otro
Name: provincia, dtype: category
Categories (3, object): [Concepcion, Otro, Santiago]

In [24]:
cot_mod = cot_mod.drop(['Provincia'], axis=1)

### 3.1.8.Comuna
Esta variable indica si el cliente facilito el correo en alguna cotizacion

In [25]:
cot_all['Comuna'].unique()

array(['Concepción', 'Chillán', 'San Pedro de la Paz', 'Talcahuano',
       'La Florida', 'Hualpén', 'Santiago', 'Constitución', 'Los Ángeles',
       'Ñuñoa', 'Providencia', 'Penco', 'Tomé', 'Buin', 'Antofagasta',
       'Ovalle', 'Puerto Montt', 'Punta Arenas', 'Chiguayante',
       'El Bosque', 'Yungay', 'Florida', 'Calama', 'Lota', 'Coronel',
       'Isla de Pascua', 'Viña del Mar', 'Machalí', 'Arauco',
       'Las Condes', 'Vitacura', 'Pirque', 'Contulmo', 'Concón',
       'Santa Juana', 'Puerto Varas', 'Cañete', 'Huechuraba', 'Valdivia',
       'Coyhaique', 'Colbún', 'Osorno', 'San Felipe', 'Iquique',
       'Rancagua', 'Temuco', 'Cochrane', 'Chillán Viejo', 'Maipú',
       'Aysén', 'Villarrica', 'La Reina', 'Peñalolén', 'Estación Central',
       'Colina', 'Quilicura', 'Curanilahue', 'Copiapó', 'Algarrobo',
       'Talca', 'San Carlos', 'Mulchén', 'Coelemu', 'Angol', 'Laja',
       'Nacimiento', 'Arica', 'La Serena', 'Hualqui', 'San Miguel',
       'Pelluhue', 'Quillón', 'Valpar

In [26]:
cot_mod['comuna'] = cot_all['Comuna'].copy()

cot_mod['comuna'] = cot_mod['comuna'].replace(
    ['Ovalle', 'Constitución','Tomé', 'Chillán','Yungay',
     'Puerto Montt', 'Santiago', 'El Bosque', 'La Florida','Las Condes',
     'Ñuñoa', 'Providencia', 'Buin','Florida','Viña del Mar',
     'Machalí', 'Vitacura', 'Arauco', 'Contulmo', 'Concón',
     'Santa Juana', 'Puerto Varas', 'Cañete', 'Huechuraba', 'Valdivia',
     'Coyhaique', 'Colbún', 'Osorno', 'San Felipe', 'Iquique',
     'Rancagua', 'Lota', 'Temuco', 'Cochrane', 'Chillán Viejo', 'Maipú',
     'Aysén', 'Villarrica', 'La Reina', 'Peñalolén', 'Estación Central',
     'Colina', 'Quilicura', 'Curanilahue', 'Algarrobo', 'Talca',
     'San Carlos', 'Punta Arenas', 'Mulchén', 'Coelemu', 'Angol',
     'Laja', 'Nacimiento', 'Arica', 'La Serena', 'Hualqui',
     'San Miguel', 'Calama', 'Pelluhue', 'Quillón', 'Valparaíso',
     'Llanquihue', 'Los Álamos', 'Renca', 'Santa Cruz', 'Panguipulli',
     'Cabrero', 'Cerrillos', 'Pudahuel', 'Copiapó', 'Natales', 'Tirúa',
     'Cauquenes', 'La Cisterna', 'Yumbel', 'Ránquil', 'Padre Las Casas',
     'Las Cabras', 'Coquimbo', 'Lebu', 'San José de Maipo', 'San Ramón',
     'Curicó', 'Independencia', 'Lampa ', 'Castro', 'Punitaqui',
     'Conchalí', 'San Rosendo', 'Alto Hospicio', 'Illapel', 'Huasco',
     'Collipulli', 'Tucapel', 'Alto Biobío','Pirque','Lampa','Isla de Pascua', 'Arica'],'Otro'
)

In [27]:
cot_mod['comuna'] = cot_mod['comuna'].astype('category')
cot_mod['comuna'].head()

0    Concepción
1    Concepción
2    Concepción
3    Concepción
4          Otro
Name: comuna, dtype: category
Categories (10, object): [Antofagasta, Chiguayante, Concepción, Coronel, ..., Otro, Penco, San Pedro de la Paz, Talcahuano]

In [28]:
cot_mod = cot_mod.drop(['Comuna'], axis=1)

### 3.1.9.Atributos Provistos
Esta variable indica el numero de veces que el cliente ha hecho cotizaciones

In [29]:
cot_mod['is_nombre'] = ~cot_all['Nombre'].isnull()
cot_mod['is_apellido1'] = ~cot_all['Apellido 1'].isnull()
cot_mod['is_apellido2'] = ~cot_all['Apellido 2'].isnull()
cot_mod['is_nombre_completo'] = ~cot_all['Nombre Completo'].isnull()
cot_mod['is_correo'] = ~cot_all['Correo Electronico'].isnull()
cot_mod['is_telefono'] = ~cot_all['Telefono'].isnull()
cot_mod['is_actividad'] = ~cot_all['Actividad'].isnull()
cot_mod['is_estado_civil'] = ~cot_all['Estado Civil'].isnull()

cot_mod = cot_mod.drop(['Telefono','Correo Electronico','Telefono','Nombre','Apellido 1','Apellido 2','Nombre Completo'], axis=1)

### 3.1.11.Negocio
Indica si el cliente siguio el proceso de cotizacion y lo transformo en un proceso de negocio

In [30]:
# ruts de los clientes en negocio
neg_ids = neg_all['ID Cotizacion'].unique().tolist()

# cot_neg = cot_all.copy()
cot_mod['Negocio'] = cot_all['ID'].isin(neg_ids)
cot_mod[cot_mod['Negocio'] == True].shape

(2336, 39)

In [31]:
# negocio = []
# for rut in cot_all['RUT'].tolist():
#     if rut in neg_ruts:
# #         print(rut)
#         df = neg_all[neg_all['RUT']==rut]
#         negocio.append(df['Proyecto'])

# print(negocio)
# print(neg_all.shape[0])
# print("Nro Filas en cot que tienen ruts en neg :",cot_mod.loc[cot_neg['Negocio']==True].shape[0])

### 3.2. Extraccion de Datos
Esta variable indica el numero de veces que el cliente ha hecho cotizaciones

In [32]:
# Construir '#cotizaciones' a partir del nro de filas donde participa el mismo rut
nro_cot_depto = []
nro_cot_esta = []
nro_cot_estu = []
nro_cot_bod = []

mean_cot_depto = []
mean_cot_esta = []
mean_cot_estu = []
mean_cot_bod = []

ruts = []
ruts_nro_cot = []
rut_reg = []
rut_prov = []
rut_com = []

nro_proyectos = []

negocio = []

cot_ruts = pd.DataFrame()

for group, frame in  cot_mod.groupby('RUT'):

    #Check , debe ir a data exploration
#     print(frame['Sexo'].unique())
    if len(frame['Proyecto'].unique()) > 1:
        print('nombre ',group, frame['Proyecto'].unique(), frame['Negocio'].unique())
#     if len(frame['is_correo'].unique()) > 1:
#         print('correo ',group)
#         print(group)
#     if len(frame['Ejecutivo Comercial'].unique()) > 1:
#         print('JC ',group)
#         print(group)
    
    nro_proyectos.append(len(frame['Proyecto'].unique()))
    negocio.append(frame['Negocio'].tolist()[0])
    
    ruts.append(group)
    rut_reg.append(frame['region'].tolist()[0])
    rut_prov.append(frame['provincia'].tolist()[0])
    rut_com.append(frame['comuna'].tolist()[0])
    
    nro_cot_depto.append(frame['#vivienda'].sum())
    nro_cot_bod.append(frame['#bodega'].sum())
    nro_cot_esta.append(frame['#estacionamiento'].sum())
    nro_cot_estu.append(frame['#estudio'].sum())
    
    mean_cot_depto.append(frame['#vivienda'].mean())
    mean_cot_bod.append(frame['#bodega'].mean())
    mean_cot_esta.append(frame['#estacionamiento'].mean())
    mean_cot_estu.append(frame['#estudio'].mean())
    ruts_nro_cot.append(frame['ID'].count())

nombre  000000000-0 ['San Andres Del Valle' 'Edificio Urban 1470'] [False]
nombre  10016093-5 ['Edificio Junge' 'San Andres Del Valle'] [ True False]
nombre  10029258-0 ['San Andres Del Valle' 'Edificio Urban 1470'] [False]
nombre  10037203-7 ['San Andres Del Valle' 'Edificio Urban 1470'] [False]
nombre  10171120-k ['Edificio Junge' 'San Andres Del Valle'] [False]
nombre  10255699-2 ['Altos del Valle' 'San Andres Del Valle'] [False]
nombre  10266404-3 ['Edificio Junge' 'Edificio Mil610'] [False]
nombre  10278036-1 ['Altos del Valle' 'Edificio Junge' 'Edificio Mil610'] [False]
nombre  10292510-6 ['Edificio Mil610' 'Edificio Urban 1470'] [False]
nombre  10444048-7 ['Altos del Valle' 'San Andres Del Valle'] [False  True]
nombre  10516915-9 ['Altos del Valle' 'San Andres Del Valle'] [False  True]
nombre  10540790-4 ['Altos del Valle' 'San Andres Del Valle'] [False  True]
nombre  10548356-2 ['Edificio Junge' 'San Andres Del Valle'] [False]
nombre  10552598-2 ['Altos del Valle' 'Edificio Urb

nombre  15180599-k ['Edificio Mil610' 'San Andres Del Valle'] [False]
nombre  15183559-7 ['Edificio Mil610' 'Edificio Urban 1470'] [False]
nombre  15183911-8 ['San Andres Del Valle' 'Edificio Urban 1470'] [False]
nombre  15185480-k ['Edificio Mil610' 'Edificio Urban 1470'] [False]
nombre  15290575-0 ['Edificio Mil610' 'San Andres Del Valle'] [False]
nombre  15328933-6 ['Altos del Valle' 'Edificio Urban 1470'] [False]
nombre  15352430-0 ['Edificio Mil610' 'Edificio Urban 1470'] [False]
nombre  15373272-8 ['Altos del Valle' 'San Andres Del Valle'] [ True False]
nombre  15380835-k ['Edificio Mil610' 'Edificio Urban 1470'] [False]
nombre  15497522-5 ['Edificio Mil610' 'San Andres Del Valle'] [False]
nombre  15592423-3 ['San Andres Del Valle' 'Edificio Urban 1470'] [False  True]
nombre  15600167-8 ['Edificio Mil610' 'San Andres Del Valle'] [False  True]
nombre  15608715-7 ['Altos del Valle' 'Edificio Urban 1470'] [False  True]
nombre  15612765-5 ['Edificio Mil610' 'San Andres Del Valle'] [F

nombre  17206016-1 ['Edificio Junge' 'San Andres Del Valle'] [False]
nombre  17206615-1 ['Edificio Mil610' 'Edificio Urban 1470'] [False]
nombre  17207987-3 ['Edificio Mil610' 'San Andres Del Valle'] [False]
nombre  17221033-3 ['Edificio Mil610' 'San Andres Del Valle'] [False  True]
nombre  17221314-6 ['Edificio Junge' 'Edificio Urban 1470'] [False]
nombre  17224305-3 ['San Andres Del Valle' 'Edificio Urban 1470'] [False  True]
nombre  17236737-2 ['Altos del Valle' 'Edificio Urban 1470'] [False]
nombre  17343482-0 ['Edificio Mil610' 'San Andres Del Valle'] [False]
nombre  17343976-8 ['San Andres Del Valle' 'Edificio Urban 1470'] [False]
nombre  17344468-0 ['Edificio Junge' 'Edificio Mil610'] [False]
nombre  17344691-8 ['Altos del Valle' 'Edificio Urban 1470'] [ True False]
nombre  17344866-k ['San Andres Del Valle' 'Edificio Urban 1470'] [False]
nombre  17345100-8 ['Edificio Mil610' 'San Andres Del Valle'] [False  True]
nombre  17345529-1 ['Edificio Mil610' 'San Andres Del Valle' 'Edif

### 3.1.10.Tiempo entre Cotizaciones
Esta variable indica el numero de veces que el cliente ha hecho cotizaciones

In [33]:
# Construir 'promedio_de_tiempo' entre cotizacones donde participa el mismo rut

# 4.Integrate Data

In [34]:
list_cols = [ruts,rut_reg,rut_prov,rut_com,nro_proyectos,nro_cot_bod, nro_cot_depto, nro_cot_esta, nro_cot_estu, mean_cot_bod,
             mean_cot_depto,mean_cot_esta,mean_cot_estu, negocio]
list_labels = ['rut','loc_region','loc_provincia','loc_comuna','nro_proyectos','nro_cot_bod', 'nro_cot_depto', 'nro_cot_esta', 'nro_cot_estu', 'mean_cot_bod',
             'mean_cot_depto','mean_cot_esta','mean_cot_estu','negocio']

zipped = list(zip(list_labels,list_cols))
data = dict(zipped)
personas = pd.DataFrame(data)

personas

,loc_comuna,loc_provincia,loc_region,mean_cot_bod,mean_cot_depto,mean_cot_esta,mean_cot_estu,negocio,nro_cot_bod,nro_cot_depto,nro_cot_esta,nro_cot_estu,nro_proyectos,rut
0,Concepción,Concepcion,Bio-Bio,0.000000,1.000000,0.000000,0.0,False,0,1,0,0,1,6180535
1,Concepción,Concepcion,Bio-Bio,0.000000,1.000000,0.000000,0.0,False,0,1,0,0,1,9559524
2,Concepción,Concepcion,Bio-Bio,0.000000,1.000000,0.000000,0.0,False,0,1,0,0,1,156778804
3,Concepción,Concepcion,Bio-Bio,0.500000,1.000000,0.500000,0.0,False,1,2,1,0,2,000000000-0
4,Concepción,Concepcion,Bio-Bio,1.000000,1.000000,1.000000,1.0,False,2,2,2,2,1,10000513-1
5,Concepción,Concepcion,Bio-Bio,1.000000,1.000000,1.000000,0.0,True,2,2,2,0,1,10000522-0
6,Concepción,Concepcion,Bio-Bio,1.000000,1.000000,1.000000,0.0,False,2,2,2,0,1,10005853-7
7,Otro,Otro,Otro,0.000000,1.000000,0.000000,0.0,False,0,2,0,0,1,10007663-2
8,Otro,Concepcion,Bio-Bio,0.000000,1.000000,1.000000,0.0,False,0,2,2,0,1,10011197-7
9,Concepción,Concepcion,Bio-Bio,0.000000,1.000000,0.000000,0.0,False,0,1,0,0,1,10014004-7


In [35]:
personas.loc_provincia.unique()

array(['Concepcion', 'Otro', 'Santiago'], dtype=object)

In [36]:
personas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6184 entries, 0 to 6183
Data columns (total 14 columns):
loc_comuna        6184 non-null object
loc_provincia     6184 non-null object
loc_region        6184 non-null object
mean_cot_bod      6184 non-null float64
mean_cot_depto    6184 non-null float64
mean_cot_esta     6184 non-null float64
mean_cot_estu     6184 non-null float64
negocio           6184 non-null bool
nro_cot_bod       6184 non-null int64
nro_cot_depto     6184 non-null int64
nro_cot_esta      6184 non-null int64
nro_cot_estu      6184 non-null int64
nro_proyectos     6184 non-null int64
rut               6184 non-null object
dtypes: bool(1), float64(4), int64(5), object(4)
memory usage: 634.2+ KB


In [37]:
# Transformar todo en un data set

# 5.Format Data

In [38]:
personas['loc_comuna'] = personas['loc_comuna'].astype('category')
personas['loc_provincia'] = personas['loc_provincia'].astype('category')
personas['loc_region'] = personas['loc_region'].astype('category')

In [39]:
personas.to_csv(data_directory+'personas.csv')